In [14]:
import numpy as np
from random import seed
from random import random
from math import exp
import tensorflow as tf
import sklearn as skl
from sklearn.metrics import f1_score

In [15]:
inputFilenameWithPath = 'train_data.txt'
testFilenameWithPath = 'test_data.txt'
inputData = np.loadtxt(inputFilenameWithPath, delimiter=",")
testData = np.loadtxt(testFilenameWithPath, delimiter=",")
n_inputs = len(inputData[0]) - 1
n_outputs = len(set([row[-1] for row in inputData]))
n_neurons = 3;
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name = "X")
#y = tf.placeholder(tf.float32, shape=(None, 1), name = "y")
y = tf.placeholder(tf.int32, [None])
y_one_hot = tf.one_hot( y , 2 )

In [16]:
def predict(row):
	return np.argmax(row, axis=0)


In [17]:
def build_network():
    learning_rate = 0.01
    intLabelling = np.vectorize(lambda x: int(x))
    softmax_op = np.vectorize(lambda data: predict(data))
    with tf.name_scope("dnn"):
        hidden1 = tf.layers.dense(X,n_neurons, name="hidden1",activation=tf.nn.sigmoid)
        hidden2 = tf.layers.dense(hidden1,n_neurons, name="hidden2",activation=tf.nn.sigmoid)
        logits = tf.layers.dense(hidden2,n_outputs, name="outputs")
    with tf.name_scope("loss"):
        xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=y_one_hot, logits = logits)
        loss = tf.reduce_mean(xentropy, name="loss")
    with tf.name_scope("train"):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        training_op = optimizer.minimize(loss)
    with tf.name_scope("eval"):
        correct = tf.nn.in_top_k(logits, y, 1)
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    init = tf.global_variables_initializer()
    n_iterations = 50
    batch_size = 10
    n_epochs = 50
    with tf.Session() as sess:
        init.run()
        for epoch in range(n_epochs): #For each epoch
            for i in range(n_iterations):
                random_index = np.random.randint(n_iterations)
                inputRows = inputData[random_index:random_index+batch_size,:]
                #print(inputRow)
                labels = intLabelling(inputRows[:,2])
                sess.run(training_op, feed_dict={X:inputRows[:,:2], y:labels})
            acc_train = accuracy.eval(feed_dict={X:inputRows[:,:2], y:labels})
            epoch_loss = loss.eval(feed_dict={X:inputRows[:,:2], y:labels})
            print('Epoch=%d, Train accuracy:=%.3f , Loss:=%.3f' % (epoch, acc_train, epoch_loss))
            
        
        #Now make the predictions with the trained network
        predictions = logits.eval(feed_dict={X:testData[:,:2]})
        trueLabels = intLabelling(testData[:,2])
        acc_test = accuracy.eval(feed_dict={X:predictions, y:trueLabels})
        print("Test accuracy:",acc_test)
        return predictions

        

In [18]:
#Call the method to invoke the whole training and test workflows
predictions = build_network()




Epoch=0, Train accuracy:=0.500 , Loss:=0.742
Epoch=1, Train accuracy:=0.500 , Loss:=0.727
Epoch=2, Train accuracy:=0.500 , Loss:=0.719
Epoch=3, Train accuracy:=0.500 , Loss:=0.711
Epoch=4, Train accuracy:=0.500 , Loss:=0.703
Epoch=5, Train accuracy:=0.500 , Loss:=0.694
Epoch=6, Train accuracy:=0.500 , Loss:=0.684
Epoch=7, Train accuracy:=0.500 , Loss:=0.677
Epoch=8, Train accuracy:=0.500 , Loss:=0.670
Epoch=9, Train accuracy:=0.800 , Loss:=0.666
Epoch=10, Train accuracy:=0.900 , Loss:=0.658
Epoch=11, Train accuracy:=1.000 , Loss:=0.642
Epoch=12, Train accuracy:=1.000 , Loss:=0.632
Epoch=13, Train accuracy:=1.000 , Loss:=0.627
Epoch=14, Train accuracy:=1.000 , Loss:=0.620
Epoch=15, Train accuracy:=0.900 , Loss:=0.613
Epoch=16, Train accuracy:=1.000 , Loss:=0.588
Epoch=17, Train accuracy:=1.000 , Loss:=0.592
Epoch=18, Train accuracy:=1.000 , Loss:=0.562
Epoch=19, Train accuracy:=1.000 , Loss:=0.552
Epoch=20, Train accuracy:=1.000 , Loss:=0.539
Epoch=21, Train accuracy:=1.000 , Loss:=0.52

In [19]:
#print(predictions)
preds = []
for i in range(len(predictions)):
    preds.append(predict(predictions[i,:]))

preds
intLabelling = np.vectorize(lambda x: int(x))
truth = intLabelling(testData[:,2])
f1 = skl.metrics.f1_score(truth, preds, average='micro')  
precision = skl.metrics.precision_score(truth, preds, average='micro')
recall = skl.metrics.recall_score(truth, preds, average='micro')
print('\nTest Precision=%.2f' % (precision))
print('Test Recall=%.2f' % (recall))
print('Test F1 Score=%.2f' % (f1))
    


Test Precision=1.00
Test Recall=1.00
Test F1 Score=1.00
